## GPT Vision 에게 이미지 설명 요청하기

In [3]:
!pip install python-dotenv

  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.1-py3-none-any.whl (20 kB)


In [4]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [5]:
load_dotenv(dotenv_path='../.env')
api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키를 가져오기

client = OpenAI(api_key=api_key)  # 오픈AI 클라이언트의 인스턴스 생성

In [6]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "이 이미지에 대해 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": "https://images.unsplash.com/photo-1736264335247-8ec5664c8328?q=80&w=1887&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D",
                },
            },
        ],
    }
]

In [7]:
response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response

ChatCompletion(id='chatcmpl-C3k2vZC6VMW8Yak7scvv2PPaOSSuj', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='이미지에서 한 여성이 주방에서 요리하는 모습입니다. 그녀는 검은색 운동복을 입고 있으며, 휘핑기로 그릇에 담긴 계란을 저으며 웃고 있습니다. 근처에 팬에는 베이컨 같은 음식이 요리되고 있습니다. 주방의 분위기는 밝고 깔끔합니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1755008021, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_ff25b2783a', usage=CompletionUsage(completion_tokens=74, prompt_tokens=1119, total_tokens=1193, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

### 내가 가진 이미지로 설명 요청하기

In [8]:
import base64

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
image_path = "./data/sample_cat2.jpg"

# 이미지를 base64로 인코딩
base64_image = encode_image(image_path)

print(base64_image)

In [11]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "이 이미지에 대해 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}",
                },
            },
        ],
    }
]

In [12]:
response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

'이 이미지는 이색안 색소상을 가진 고양이를 보여줍니다. 고양이의 한쪽 눈은 파란색이고 다른 쪽 눈은 노란색입니다. 이런 독특한 눈 색깔은 흔히 흰색 털을 가진 고양이에서 볼 수 있습니다. 고양이는 카메라를 응시하고 있으며, 배경은 어두운 목재 바닥으로 보입니다.'

### 비교

In [13]:
first_base64 = encode_image("./data/sample_cat.jpg")
second_base64 = encode_image("./data/sample_cat2.jpg")

In [16]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "두 고양이의 차이점을 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{first_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{second_base64}",
                },
            },
        ],
    }
]

In [17]:
response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

'첫 번째 고양이와 두 번째 고양이의 차이점은 다음과 같습니다:\n\n1. **털 색상 및 패턴**: \n    - 첫 번째 고양이는 줄무늬가 있는 갈색 털을 가지고 있습니다.\n    - 두 번째 고양이는 전체적으로 흰색 털을 가지고 있습니다.\n\n2. **눈 색깔**:\n    - 첫 번째 고양이는 녹색 눈을 가지고 있습니다.\n    - 두 번째 고양이는 오드아이로, 한쪽 눈은 파란색이고 다른 쪽 눈은 노란색입니다.\n\n이러한 차이점 외에도 개체의 표정이나 자세 등에서 차이가 있을 수 있습니다.'

### GPT 비전의 한계 살펴보기

In [21]:
Foecd_rnd_2021_base64 = encode_image("./data/oecd_rnd_2021_large.png")
oecd_rnd_2022_base64 = encode_image("./data/oecd_rnd_2022_large.png")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "첫번째는 2021년 데이터이고, 두번째는 2022년 데이터입니다. 이 데이터에 대해 설명해주세요. 어떤 변화가 있었나요? 한국 중심으로 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2021_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2022_base64}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

'두 그래프는 2021년과 2022년의 각국 연구개발비와 GDP 대비 연구개발비 비중을 나타내고 있습니다. 한국을 중심으로 분석해보겠습니다.\n\n1. **연구개발비**:\n   - **2021년**: 약 89,282백만 달러\n   - **2022년**: 약 91,013백만 달러\n   - 연구개발비가 증가하였습니다.\n\n2. **GDP 대비 연구개발비 비중**:\n   - **2021년**: 4.93%\n   - **2022년**: 4.93% (백분율 변화 없음)\n   - 비중은 동일하게 유지되었습니다.\n\n3. **총체적 변화**:\n   - 연구개발비는 증가했지만, GDP 대비 비중은 변동이 없습니다.\n   - 이는 GDP 증가와 연구개발비 증가가 비슷한 비율로 이루어졌다는 의미입니다.\n\n이 변화를 통해 한국은 일정 수준의 연구개발 투자를 지속적으로 유지하고 있음을 알 수 있습니다. 이러한 지속적 투자는 기술 발전 및 경제 성장에 긍정적인 영향을 미칠 가능성이 큽니다.'

#### 2022년 한국은 87225 백만 달러인데, 영국의 값을 한국으로 잘못 착각하고 있다.

##### [추가] gpt-5 로 시도해보자

In [23]:
response = client.chat.completions.create(
    model="gpt-5",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

'요약\n- 기준: OECD MSTI, 금액은 달러 환산(2021: 1,143.96원/$ → 2022: 1,291.45원/$).\n- 막대=연구개발비(백만 달러), 점=GDP 대비 연구개발비 비중.\n\n한국 중심 변화\n- 절대 규모(달러): 89,282M$ → 87,225M$로 약 −2.3% 감소.\n  - 원화가 약세(환율 상승)여서 달러 표시액이 줄어든 영향이 큼. 원화 기준으로는 증가했을 가능성이 높습니다.\n- 세계 순위(규모): 5위 → 6위.\n  - 영국(2021년 자료가 대폭 상향·개편) 91,013M$에 밀려 한 계단 하락.\n- R&D 집약도(GDP 대비 비중): 4.93% → 5.21%로 +0.28%p 상승.\n  - OECD 최상위권 유지(이스라엘 5.56% 다음 수준). “규모 대비 투자강도”는 오히려 강화됨.\n\n주요국 비교 맥락\n- 미국: 720.9B$ → 806.0B$(+11.8%), 비중 3.45%→3.46%(유지).\n- 중국: 353.5B$ → 433.5B$(+22.6%), 비중 2.40%→2.43%(소폭↑).\n- 일본: 165.0B$ → 165.0B$(보합), 비중 3.27%→3.30%(소폭↑).\n- 독일: 121.7B$ → 133.9B$(+10%), 비중 3.13%(유지).\n- 영국: 49.2B$ → 91.0B$(대폭↑), 비중 1.71%→2.91%.\n  - 영국의 통계 방식/추정치 개편 영향으로 한국의 “규모 순위” 하락에 직접적 요인.\n\n읽을 때 유의할 점\n- 국가별 최신 연도가 서로 달라(한국 2022, 미국·중국 등 2021) 순위가 추후 변동될 수 있음.\n- 환율 차이가 크므로 달러 기준 증감과 국내 통화 기준 증감이 다를 수 있음.\n\n정리\n- 한국은 달러 기준 총액은 소폭 줄었지만, GDP 대비 비중은 5%를 넘겨 투자 강도가 더 높아졌습니다.\n- 절대 규모 순위는 영국의 통계 상향으로 5위→6위로 내려갔으나, 연구개발 집약도 측면에서는 세계 최상위 수준을 유지·강화했습니다.'

gpt-5 는 한국을 잘 찾는듯

In [25]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5",
    input="How much gold would it take to coat the Statue of Liberty in a 1mm layer? response in korean.",
    reasoning={
        "effort": "minimal"
    }
)

print(response)

Response(id='resp_689b51e10c1881909e3d63eb5fac8fa8008e3a3db5b78160', created_at=1755009505.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_689b51e20ff88190a85c8fceda89b053008e3a3db5b78160', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseOutputMessage(id='msg_689b51e255b48190a8f5b0ccc6e0a59b008e3a3db5b78160', content=[ResponseOutputText(annotations=[], text='대략적인 계산으로, 자유의 여신상 전체를 1mm 두께의 금으로 도금하려면 약 50~60톤의 금이 필요합니다.\n\n간단한 근거:\n- 자유의 여신상(토치까지)의 높이는 약 46m(동상 자체)이고, 구조를 단순화해 표면적을 약 2,500~3,000 m² 정도로 추정할 수 있습니다(큰 조각상들의 일반적인 표면적 추정치와 유사).\n- 도금 두께 1mm = 0.001 m → 필요한 금의 부피 = 표면적 × 두께 ≈ 2,500~3,000 m² × 0.001 m = 2.5~3.0 m³\n- 금의 밀도 ≈ 19,320 kg/m³\n- 질량 ≈ 2.5~3.0 m³ × 19,320 kg/m³ = 약 48,000~58,000 kg = 48~58톤\n\n참고:\n- 실제 표면적은 세부 조각(주름, 왕관, 토치 구조, 책 등) 때문에 더 클 수 있어 여유를 두면 60톤 안팎으로 잡는 것이 현실적입니다.\n- 금 가격 변동에 따라 비용은 크게 달라지나, 톤당 수천만 달러 수준으로 총 수십